In [ ]:
import numpy as np
import tensorflow as tf
import math
import dataset_input
import resnet
import sys
import os
import pickle
from collections import Counter
import utilities
import json
from UniversalPerturb import UniversalPert
from PerImgPerturb import PerImgPert
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)




#load the testing data
version = sys.version_info
def _load_datafile(filename):
        with open(filename, 'rb') as fo:
            if version.major == 3:
                data_dict = pickle.load(fo, encoding='bytes')
            else:
                data_dict = pickle.load(fo)

            assert data_dict[b'data'].dtype == np.uint8
            image_data = data_dict[b'data']
            image_data = image_data.reshape((10000, 3, 32, 32)).transpose(0, 2, 3, 1)
            return image_data, np.array(data_dict[b'labels'])
eval_filename = 'test_batch'
path = "cifar-10-batches-py"
eval_images, eval_labels = _load_datafile(
            os.path.join(path, eval_filename))
eval_images = np.reshape(eval_images, (10000, 3072))





BINARY_SEARCH_STEPS = 12  # number of times to adjust the constant with binary search
MAX_ITERATIONS = 101  # number of iterations to perform gradient descent
#ABORT_EARLY = True       # if we stop improving, abort gradient descent early
LEARNING_RATE = 1e-2     # larger values converge faster to less accurate results
INITIAL_CONST = 0.01     # the initial constant lambda to pick as a first guess




num_class = 10

#randomly select a small amount of data
idxx = np.arange(0 , 10000)
judge_class = 5*np.ones(10)
count_num = 0
idx = np.zeros(50, dtype=int)
for nn in range(0, num_class):
    for ii in idxx:
        if judge_class[nn] == 0:
            break
        if eval_labels[ii] == nn:
            idx[count_num] = int(ii)
            count_num = count_num + 1
            judge_class[nn] = judge_class[nn] - 1



data_shuffle = eval_images[idx]
labels_shuffle = eval_labels[idx]
for jj in range(0,1):
    path_new = 'models/'+'clean'+str(jj+1)+'/'


    data_num_labels = []
    out_save_poison = []
    bestarea_cand_pos = []
    area_cand_pos = []
    out_cand_pos = []
    #pos_mul = []
    pos_single = []
    for targets in range(0, num_class):
        tf.reset_default_graph() #avoid the variable reusing
        y_tru_val = tf.convert_to_tensor(labels_shuffle)

        #find images belonging to the selected label and images that are not
        indices = tf.where(tf.not_equal(y_tru_val, targets))
        indices2 = tf.where(tf.equal(y_tru_val, targets))
        indices = tf.reshape(indices, [-1])
        indices2 = tf.reshape(indices2, [-1])
        with tf.Session() as sess:
            sess.run(tf.initializers.global_variables())
            indices = indices.eval()
            indices2 = indices2.eval()
        #sess.close
        batchsize = len(indices)
        batchsize2 = len(indices2)
        labels_shuffle_new = np.zeros((50, num_class))
        labels_shuffle_new[np.arange(50), labels_shuffle] = 1
        imgs = data_shuffle[indices]
        imgs = imgs.reshape((batchsize, 32, 32, 3))
        labs = labels_shuffle_new[indices]
        imgs2 = data_shuffle[indices2]
        imgs2 = imgs2.reshape((batchsize2, 32, 32, 3))
        labs2 = labels_shuffle_new[indices2]
        data_num_labels.append(batchsize2)   
        image = np.concatenate((imgs, imgs2), axis=0)
        with tf.Session() as sess:
            sess.run(tf.initializers.global_variables())
            regul = "l1"
            print("- regularization is" + str(regul))
            print("- trojan pixel, moving label:\t\t{}".format(targets))
            config_dict = utilities.get_config('config.json')

            model_dir = config_dict['model']['output_dir']
            if not os.path.exists(model_dir):
                os.makedirs(model_dir)

            # keep the configuration file with the model for reproducibility
            with open(os.path.join(model_dir, 'config.json'), 'w') as f:
                json.dump(config_dict, f, sort_keys=True, indent=4)

            config = utilities.config_to_namedtuple(config_dict)
            pert_p, coff_p, ind1, ind2, output_p, log_output, area = UniversalPert(sess, config, path_new, batchsize, batchsize2, regul).attack(image, labs, labs2)
            print(labs[ind1])
            print(labs2[ind2])
            #pos_mul.append(output_p)
            bestarea_cand_pos.append(pert_p)
            out_cand_pos.append(log_output)
            area_cand_pos.append(area)


        temp_out_poison = []
        tf.reset_default_graph()
        with tf.Session() as sess:
            sess.run(tf.initializers.global_variables())
            config_dict = utilities.get_config('config.json')
            regul = 'l1'
            model_dir = config_dict['model']['output_dir']
            if not os.path.exists(model_dir):
                os.makedirs(model_dir)

            # keep the configuration file with the model for reproducibility
            with open(os.path.join(model_dir, 'config.json'), 'w') as f:
                json.dump(config_dict, f, sort_keys=True, indent=4)

            config = utilities.config_to_namedtuple(config_dict)
            image_new = np.reshape(imgs, (batchsize, 32, 32, 3))
            lab_new = np.reshape(labs2[0], (1, 10))
            pert_single, coff_single, ind_single, output_single = PerImgPert(sess, config, path_new, batchsize, regul).attack(image_new, np.repeat(lab_new, batchsize, axis=0))
            pos_single.append(output_single)
        for i in range(batchsize):
            temp_single = np.dot(output_p[i], np.transpose(output_single[i]))/(np.linalg.norm(output_p[i])*np.linalg.norm(output_single[i]))
            temp_out_poison.append(temp_single)
        out_save_poison.append(temp_out_poison)
        
    best_area = np.amin(bestarea_cand_pos)
    for cla in range(num_class):
        temp_best = np.quantile(out_save_poison[cla], 0.5)
        output_p = out_cand_pos[cla]
        area = area_cand_pos[cla]
        for kk in range(len(area)):
            temp_out_poison = []
            if area[kk] < 1.1*best_area:        
                for i in range(len(pos_single[cla])):
                    temp_single = np.dot(output_p[kk][i], np.transpose(pos_single[cla][i]))/(np.linalg.norm(output_p[kk][i])*np.linalg.norm(pos_single[cla][i]))
                    temp_out_poison.append(temp_single)
                if np.quantile(temp_out_poison, 0.25) > temp_best:
                    out_save_poison[cla] = temp_out_poison[:]
                    
#Save your detection results                    
#     res_dir = 'result/'+'pattern'+str(jj+1)+'.pkl'
#     with open(res_dir, 'wb') as f:  # Python 3: open(..., 'wb')
#         pickle.dump(out_save_poison, f)


#check whether the model is a Trojan model and the target label
T = 0.75 #a preset threshold
target = -1
for i in range(10):
    if np.quantile(out_save_poison[i], 0.25) > T:
        print('The model is a Trojan model and the target label is: {}'.format(i))
        target = i
if target == -1:
    print('The model is a clean model')